In [ ]:
%%html
<script type=text/javascript>
/* Add a button for showing or hiding input */
on = "Show input";
off = "Hide input";
function onoff(){
  currentvalue = document.getElementById('onoff').value;
  if(currentvalue == off){
    document.getElementById("onoff").value=on;
      $('div.input').hide();
  }else{
    document.getElementById("onoff").value=off;
      $('div.input').show();
  }
}

/* Launch first notebook cell on start */
function launch_first_cell (evt) {
  if (!launch_first_cell.executed
      && Jupyter.notebook.kernel
  ) {
    Jupyter.notebook.get_cells()[0].execute();
    launch_first_cell.executed = true;
  }
}
$(document).ready(function (){onoff();})
</script>

<p>Press this button to show/hide the code used in the notebook:
<input type="button" class="ui-button ui-widget ui-state-default \
ui-corner-all ui-button-text-only" value="Hide input" id="onoff" \
onclick="onoff();"></p>


# Data processing, skip further for discussion.

In [ ]:
%matplotlib inline
from matplotlib import pyplot
import pandas

In [ ]:
problem_names = {  # Conversion to meangingful problem names.
}

question_scores = {  # Numerical scores for each option
}

In [ ]:
data = pandas.read_pickle('test_results.pickle')
data = data.T

# Remove individual remarks and unused feedback
grades = data.T[~data.columns.str.contains('remarks')].T
grades = grades.T[grades.sum() != 0].T.astype(int)

scores = grades.columns.map(lambda name: question_scores[name])

# Make feedback human-readable
grades.columns = grades.columns.str.replace(':', ': ')
for name, new_name in problem_names.items():
    grades.columns = grades.columns.str.replace(name, new_name)

count = grades.sum().astype(int)
summary = pandas.DataFrame({'frequency': count, 'points': scores}, index=grades.columns)

# How many times each feedback option was received

In [ ]:
summary

In [ ]:
points = grades * scores

# Score discributions per problem and total

In [ ]:
for problem in problem_names.values():
    grades[problem] = points.T[points.columns.str.startswith(problem)].sum()
    grades[problem] = grades[problem].apply(lambda i: max(i, 0))

grades['total'] = grades[sorted(problem_names.values())].sum(axis=1)

grades[sorted(problem_names.values()) + ['total']].describe().round(1)

# Correlations between different problems.

In [ ]:
grades[sorted(problem_names.values()) + ['total']].corr().round(1)

We see that the questions are mostly independent, except for 2a being related to 2b, which is reasonable.

# Finally: the score distribution

In [ ]:
(grades['total'] / 10).hist(bins=40, grid=False)
pyplot.xlim((0, 10))
pyplot.xlabel('Score');

The test was quite straightforward: the median is around 8, and a lot of people managed to score a full 10.

In [ ]:
grades.to_pickle('grades.pickle')

grades.to_excel('test.xlsx')